In [1]:
import pandas as pd
import numpy as np
import nltk.tokenize
import itertools

from pathlib import Path

from fastai import *
from fastai.text import *

In [2]:
model_title = '2.0-ULMFiT'
MODEL_PATH = Path(f'../data/models/{model_title}')
MODEL_PATH.mkdir(exist_ok=True)

In [3]:
DOWNLOAD_WIKITEXT = False
DOWNLOAD_PRETRAINED_LYRICS = False

In [4]:
if DOWNLOAD_WIKITEXT:
    url = 'http://files.fast.ai/models/wt103_v1/'
    download_url(f'{url}lstm_wt103.pth', MODEL_PATH/'models/lstm_wt103.pth')
    download_url(f'{url}itos_wt103.pkl', MODEL_PATH/'models/itos_wt103.pkl')
    
if DOWNLOAD_PRETRAINED_LYRICS:
    url = 'https://storage.googleapis.com/capstone-deep-lyrics/'
    download_url(f'{url}2.0-ULMFiT.pth', MODEL_PATH/'models/2.0-ULMFiT.pth')
    download_url(f'{url}itos.pkl', MODEL_PATH/'models/itos.pkl')

In [5]:
data_lm = TextLMDataBunch.from_tokens(MODEL_PATH,
                                      bs=128,
                                      max_vocab=10000)

print(data_lm.train_ds.vocab_size)

10002


In [6]:
x,y = next(iter(data_lm.train_dl))
example = x[:20,:10].cpu()
texts = pd.DataFrame([data_lm.train_ds.vocab.textify(l).split(' ') for l in example])
print(x.shape, y.shape)
texts

torch.Size([95, 128]) torch.Size([12160])


,0,1,2,3,4,5,6,7,8,9
0,xbos,xeol,'s,[outro],will,xeol,so,xbol,won,my
1,xbol,xbol,sad,xeol,destroy,xbol,i,i,'t,soul
2,i,i,but,xbol,her,i,fight,said,deal,xeol
3,hear,'ll,you,i,xeol,'ve,xeol,",",xeol,xeos
4,you,do,seem,'ll,xbol,been,xbol,it,xbol,xbos
5,praying,anything,better,never,so,takin,i,'s,i,xbol
6,with,for,when,know,i,',cannot,my,won,xxunk
7,your,you,i,xeol,'m,all,shake,life,'t,xxunk
8,hands,",",'m,xbol,giving,the,from,xeol,change,em
9,xxunk,show,gone,i,up,blame,my,xbol,",",busca


In [7]:
learn = RNNLearner.language_model(data_lm,
                                  pretrained_fnames=['2.0-ULMFiT', 'itos'],
                                  drop_mult=0.5)

In [8]:
learn.validate()

[2.5710025, 0.4932660821494095]

In [8]:
# GPU = True

In [10]:
# def cpu_load(self, name:PathOrStr):
#     """Load model onto CPU that was trained on a GPU `name` from `self.model_dir`.
#        We need these because the fastai load function doesn't allow for a remapping of the storage location."""
#     self.model.load_state_dict(torch.load(self.path/self.model_dir/f'{name}.pth', map_location=lambda storage, loc: storage))

# setattr(RNNLearner, 'cpu_load', cpu_load) #monkey patch onto our RNNLearner

In [11]:
# if not GPU:
#     learn.cpu_load(f'{model_title}')
# else:
#     learn.load(f'{model_title}')

In [10]:
validate(learn.model, learn.data.valid_dl, learn.loss_func)

AttributeError: 'tuple' object has no attribute 'log_softmax'